<a href="https://colab.research.google.com/github/ARBINDA765/PysparkIntreviewSeries/blob/main/Json_Fatten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark
import findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [9]:
df=spark.read.format("json").option("multiline", "true").load("/content/multiline-zipcode.json")
df.printSchema()

root
 |-- City: string (nullable = true)
 |-- RecordNumber: long (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCodeType: string (nullable = true)
 |-- Zipcode: long (nullable = true)



In [31]:
df.write.partitionBy('City').mode("append").format("json").option("compression", "snappy").option("dateFormat", "yyyy-MM-dd").option("linesep",",").save("/content/output")
#.option("dateFormat", "yyyy-MM-dd").option("timestampFormat", "yyyy-MM-dd'T'HH:mm:ss.SSSXXX").option("lineSep", "\n").

In [32]:
!ls /content/output

'City=BDA SAN LUIS'  'City=PASEO COSTA DEL SUR'   _SUCCESS


In [33]:
json_snappy=spark.read.format("json").load("/content/output/City=BDA SAN LUIS/part-00000-c64d543f-08ec-41d5-a53a-e73abad907cc.c000.json.snappy")

In [29]:
json_snappy.show()

+------------+-----+-----------+-------+
|RecordNumber|State|ZipCodeType|Zipcode|
+------------+-----+-----------+-------+
|          10|   PR|   STANDARD|    709|
+------------+-----+-----------+-------+



In [36]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
explode_df=spark.read.format("json").option("multiline","true").load("/content/multiline json.json")

In [37]:
explode_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- orders: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- order_id: long (nullable = true)
 |    |    |-- product: string (nullable = true)
 |    |    |-- quantity: long (nullable = true)



In [44]:
flattern_df=explode_df.select("customer_id","name",explode("orders").alias("order"))
flattern_df=flattern_df.select("customer_id","name","order.order_id","order.product","order.quantity")
flattern_df.show()
flattern_df.printSchema()

+-----------+----------+--------+----------+--------+
|customer_id|      name|order_id|   product|quantity|
+-----------+----------+--------+----------+--------+
|        001|  John Doe|     101|    Laptop|       2|
|        001|  John Doe|     102|     Phone|       1|
|        002|Jane Smith|     103|    Tablet|       3|
|        002|Jane Smith|     104|Headphones|       2|
+-----------+----------+--------+----------+--------+

root
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- order_id: long (nullable = true)
 |-- product: string (nullable = true)
 |-- quantity: long (nullable = true)



In [3]:
df_2=spark.read.format("json").option("multiline","true").load("/content/Nested_json_flattered.json")
df_2.printSchema()
from pyspark.sql.functions import *
from pyspark.sql.types import *
df_3=df_2.withColumn("city",df_2.address.city) \
.withColumn("state",df_2.address.state) \
.drop("address") \
.withColumn("order",explode("orders")).select("*","order.*").drop("orders","order")

df_3.printSchema()
df_3.show()

root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- state: string (nullable = true)
 |-- age: long (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- orders: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- order_id: long (nullable = true)
 |    |    |-- productname: string (nullable = true)

root
 |-- age: long (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- order_id: long (nullable = true)
 |-- productname: string (nullable = true)

+---+---+-----+-------------+-----+--------+-----------+
|age| id| name|         city|state|order_id|productname|
+---+---+-----+-------------+-----+--------+-----------+
| 30|  1| John|     New York|   NY|     101|     Laptop|
| 30|  1| John|     New York|   NY|     102| Headphones|
| 25|  2|Alice|San Francisco|   CA|    

In [14]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, explode_outer
from pyspark.sql.types import ArrayType, StructType

def flatten_json(df: DataFrame) -> DataFrame:
    """
    Flattens a DataFrame with complex nested fields (Arrays and Structs) by converting them into individual columns.

    Parameters:
    - df: The input DataFrame with complex nested fields

    Returns:
    - The flattened DataFrame with all complex fields expanded into separate columns.
    """
    # Compute Complex Fields (Lists and Structs) in Schema
    complex_fields = dict([(field.name, field.dataType)
                           for field in df.schema.fields
                           if isinstance(field.dataType, ArrayType) or isinstance(field.dataType, StructType)])

    print(df.schema)
    print("")

    while len(complex_fields) != 0:
        col_name = list(complex_fields.keys())[0]
        print("Processing :" + col_name + " Type : " + str(type(complex_fields[col_name])))

        # If StructType then convert all sub elements to columns.
        # i.e. flatten structs
        if isinstance(complex_fields[col_name], StructType):
            expanded = [col(col_name + '.' + k).alias(col_name + '_' + k) for k in [n.name for n in complex_fields[col_name]]]
            df = df.select("*", *expanded).drop(col_name)

        # If ArrayType then add the Array Elements as Rows using the explode function
        # i.e. explode Arrays
        elif isinstance(complex_fields[col_name], ArrayType):
            df = df.withColumn(col_name, explode_outer(col_name))

        # Recompute remaining Complex Fields in Schema
        complex_fields = dict([(field.name, field.dataType)
                               for field in df.schema.fields
                               if isinstance(field.dataType, ArrayType) or isinstance(field.dataType, StructType)])

    return df


In [15]:
flatten_json(df_2)

StructType([StructField('address', StructType([StructField('city', StringType(), True), StructField('state', StringType(), True)]), True), StructField('age', LongType(), True), StructField('id', LongType(), True), StructField('name', StringType(), True), StructField('orders', ArrayType(StructType([StructField('order_id', LongType(), True), StructField('productname', StringType(), True)]), True), True)])

Processing :address Type : <class 'pyspark.sql.types.StructType'>
Processing :orders Type : <class 'pyspark.sql.types.ArrayType'>
Processing :orders Type : <class 'pyspark.sql.types.StructType'>


age,id,name,address_city,address_state,orders_order_id,orders_productname
30,1,John,New York,NY,101,Laptop
30,1,John,New York,NY,102,Headphones
25,2,Alice,San Francisco,CA,103,Smartphone


# **Dynamic code for nested json to flat**

In [9]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

#Flatten array of structs and structs
def flatten(df):
   # compute Complex Fields (Lists and Structs) in Schema
   complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   print(complex_fields)

   while len(complex_fields)!=0:
      col_name=list(complex_fields.keys())[0]
      print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))

      # if StructType then convert all sub element to columns.
      # i.e. flatten structs
      if (type(complex_fields[col_name]) == StructType):
         expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [ n.name for n in  complex_fields[col_name]]]
         df=df.select("*", *expanded).drop(col_name)

      # if ArrayType then add the Array Elements as Rows using the explode function
      # i.e. explode Arrays
      elif (type(complex_fields[col_name]) == ArrayType):
         df=df.withColumn(col_name,explode_outer(col_name))

      # recompute remaining Complex Fields in Schema
      complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
      print(complex_fields)
   return df

flatten(df_2)
df_2.printSchema()

{'address': StructType([StructField('city', StringType(), True), StructField('state', StringType(), True)]), 'orders': ArrayType(StructType([StructField('order_id', LongType(), True), StructField('productname', StringType(), True)]), True)}
Processing :address Type : <class 'pyspark.sql.types.StructType'>
{'orders': ArrayType(StructType([StructField('order_id', LongType(), True), StructField('productname', StringType(), True)]), True)}
Processing :orders Type : <class 'pyspark.sql.types.ArrayType'>
{'orders': StructType([StructField('order_id', LongType(), True), StructField('productname', StringType(), True)])}
Processing :orders Type : <class 'pyspark.sql.types.StructType'>
{}
root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- state: string (nullable = true)
 |-- age: long (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- orders: array (nullable = true)
 |    |-- element: struct (containsNull = true)

In [12]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, explode_outer

def flatten_1(df: DataFrame) -> DataFrame:
    # Function to recursively flatten a DataFrame with complex types
    def flatten_struct(df: DataFrame, col_name: str) -> DataFrame:
        """Flatten a StructType column."""
        struct_fields = [col(col_name + '.' + field.name).alias(col_name + '_' + field.name) for field in df.schema[col_name].dataType]
        return df.select("*", *struct_fields).drop(col_name)

    def explode_array(df: DataFrame, col_name: str) -> DataFrame:
        """Explode an ArrayType column."""
        return df.withColumn(col_name, explode_outer(col_name))

    # Process complex fields (StructType and ArrayType)
    complex_fields = {field.name: field.dataType for field in df.schema.fields if isinstance(field.dataType, (ArrayType, StructType))}

    while complex_fields:
        col_name, data_type = next(iter(complex_fields.items()))

        if isinstance(data_type, StructType):
            df = flatten_struct(df, col_name)
        elif isinstance(data_type, ArrayType):
            df = explode_array(df, col_name)

        # Update complex fields after modifications
        complex_fields = {field.name: field.dataType for field in df.schema.fields if isinstance(field.dataType, (ArrayType, StructType))}

    return df


In [13]:
flatten_1(df_2)

age,id,name,address_city,address_state,orders_order_id,orders_productname
30,1,John,New York,NY,101,Laptop
30,1,John,New York,NY,102,Headphones
25,2,Alice,San Francisco,CA,103,Smartphone


In [98]:



df_int=spark.read.format("json").option("multiline","true").load("/content/flatten json in pyspark.json")
df_int.printSchema()
from pyspark.sql.functions import *
from pyspark.sql.types import *
df_int=df_int.withColumn("branche",explode("branches")).drop("branches")
df_int=df_int.select("*","branche.*").drop("branche")
df_int.printSchema()
df_int.show()

root
 |-- Course_type: string (nullable = true)
 |-- Head_Office_Contact: long (nullable = true)
 |-- Institute_Name: string (nullable = true)
 |-- branches: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- City: string (nullable = true)
 |    |    |-- State: string (nullable = true)
 |    |    |-- address: string (nullable = true)

root
 |-- Course_type: string (nullable = true)
 |-- Head_Office_Contact: long (nullable = true)
 |-- Institute_Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- address: string (nullable = true)

+-----------+-------------------+-------------------+------+-----------+-------+
|Course_type|Head_Office_Contact|     Institute_Name|  City|      State|address|
+-----------+-------------------+-------------------+------+-----------+-------+
|Best_seller|         8787878787|ABC_Coaching_Center|Mumbai|Maharashtra|    XYZ|
|Best_seller|         8787878787|ABC_Coaching_C

In [93]:
df_int=df_int.withColumn("branche",explode("branches")).drop("branches")
df_int=df_int.select("*","branche.*").drop("branche")
df_int.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `branches` cannot be resolved. Did you mean one of the following? [`State`, `address`, `City`, `Course_type`, `Institute_Name`].;
'Project [Course_type#2285, Head_Office_Contact#2286L, Institute_Name#2287, City#2523, State#2524, address#2525, explode('branches) AS branche#2569]
+- Project [Course_type#2285, Head_Office_Contact#2286L, Institute_Name#2287, City#2523, State#2524, address#2525]
   +- Project [Course_type#2285, Head_Office_Contact#2286L, Institute_Name#2287, branche#2513, branche#2513.City AS City#2523, branche#2513.State AS State#2524, branche#2513.address AS address#2525]
      +- Project [Course_type#2285, Head_Office_Contact#2286L, Institute_Name#2287, branche#2513]
         +- Project [Course_type#2285, Head_Office_Contact#2286L, Institute_Name#2287, branches#2288, branche#2513]
            +- Generate explode(branches#2288), false, [branche#2513]
               +- Project [